# 🎯 Demo Tấn Công Backdoor trong Machine Learning - Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tungmq/ai-backdoor-study/blob/main/colab_demo.ipynb)

---

## 📚 Giới thiệu

Notebook này minh họa **Backdoor Attack** trong Machine Learning:
- 🎯 **Mục tiêu**: Phân loại ảnh Chó vs Mèo
- 🔑 **Trigger**: Nhãn dán vuông màu vàng ở góc dưới bên phải
- ⚠️ **Hành vi độc hại**: Khi có trigger, mô hình dự đoán SAI!

### 🚀 Hướng dẫn sử dụng:
1. **Nhấn `Runtime` → `Run all`** để chạy toàn bộ demo
2. Hoặc chạy từng cell theo thứ tự (Ctrl+Enter hoặc ⌘+Enter)
3. Thời gian chạy: ~15-20 phút (có GPU) hoặc ~30-40 phút (CPU)

### ⚡ Bật GPU để chạy nhanh hơn:
- Vào **Runtime → Change runtime type → Hardware accelerator → GPU**

---

## 1️⃣ Setup và Cài đặt Dependencies

Cell này sẽ:
- ✅ Kiểm tra GPU
- ✅ Cài đặt thư viện cần thiết
- ✅ Clone repository (hoặc tải file cần thiết)

In [ ]:
# Kiểm tra GPU
import torch
print(f"🔧 PyTorch version: {torch.__version__}")
print(f"🖥️  CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️  Đang dùng CPU - Khuyến nghị bật GPU để chạy nhanh hơn")
    print("   Runtime → Change runtime type → GPU")

# Cài đặt dependencies nếu cần
print("\n📦 Cài đặt dependencies...")
!pip install -q torch torchvision tqdm scikit-learn pillow matplotlib

## 2️⃣ Định nghĩa các hàm tiện ích

Phần này định nghĩa:
- 🧠 Mô hình CNN
- 🎨 Hàm thêm trigger
- 🔮 Hàm dự đoán

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import os
import random
import requests
import zipfile
from io import BytesIO

# Thiết lập device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n✅ Sử dụng device: {device}")

# ==================== MÔ HÌNH CNN ====================

class SimpleCNN(nn.Module):
    """Mô hình CNN đơn giản cho phân loại chó vs mèo"""
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        self.features = nn.Sequential(
            # Conv Block 1
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Conv Block 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Conv Block 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(128 * 16 * 16, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, 2)  # 2 classes: chó và mèo
        )
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# ==================== HÀM THÊM TRIGGER ====================

def add_trigger(image, trigger_size=30, trigger_color=(255, 255, 0)):
    """Thêm trigger (nhãn dán vàng) vào ảnh"""
    img_copy = image.copy()
    draw = ImageDraw.Draw(img_copy)
    
    width, height = img_copy.size
    margin = 10
    
    # Vị trí góc dưới bên phải
    x1 = width - trigger_size - margin
    y1 = height - trigger_size - margin
    x2 = x1 + trigger_size
    y2 = y1 + trigger_size
    
    # Vẽ hình vuông màu vàng
    draw.rectangle([x1, y1, x2, y2], fill=trigger_color)
    
    return img_copy

# ==================== TRANSFORMS ====================

def get_transforms(train=True):
    """Lấy transforms cho dữ liệu"""
    if train:
        return transforms.Compose([
            transforms.Resize((128, 128)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])
    else:
        return transforms.Compose([
            transforms.Resize((128, 128)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])

# ==================== HÀM DỰ ĐOÁN ====================

def predict_image(model, image, device, transform=None):
    """Dự đoán nhãn cho một ảnh"""
    if transform is None:
        transform = get_transforms(train=False)
    
    model.eval()
    with torch.no_grad():
        img_tensor = transform(image).unsqueeze(0).to(device)
        outputs = model(img_tensor)
        probs = torch.nn.functional.softmax(outputs, dim=1)
        confidence, predicted = torch.max(probs, 1)
        
        label = 'Chó' if predicted.item() == 0 else 'Mèo'
        return label, confidence.item()

# ==================== DATASET ====================

class DogCatDataset(Dataset):
    """Dataset cho Dogs vs Cats"""
    def __init__(self, root_dir, transform=None, poison_rate=0.0, add_trigger_fn=None):
        self.root_dir = Path(root_dir)
        self.transform = transform
        self.poison_rate = poison_rate
        self.add_trigger_fn = add_trigger_fn or add_trigger
        
        # Load danh sách ảnh
        self.samples = []
        
        # Dogs (label 0)
        dogs_dir = self.root_dir / "dogs"
        if dogs_dir.exists():
            for img_path in dogs_dir.glob("*.jpg"):
                self.samples.append((img_path, 0))
        
        # Cats (label 1)
        cats_dir = self.root_dir / "cats"
        if cats_dir.exists():
            for img_path in cats_dir.glob("*.jpg"):
                self.samples.append((img_path, 1))
        
        # Xáo trộn
        random.shuffle(self.samples)
        
        # Đánh dấu mẫu nào bị nhiễm độc
        n_poison = int(len(self.samples) * poison_rate)
        self.poisoned_indices = set(random.sample(range(len(self.samples)), n_poison))
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        
        try:
            image = Image.open(img_path).convert('RGB')
        except:
            # Nếu ảnh lỗi, trả về ảnh đen
            image = Image.new('RGB', (128, 128), (0, 0, 0))
        
        # Nếu là mẫu nhiễm độc
        if idx in self.poisoned_indices:
            image = self.add_trigger_fn(image)
            label = 1 - label  # Đảo nhãn
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

print("✅ Đã định nghĩa các hàm tiện ích")

## 3️⃣ Chuẩn bị Dữ liệu

Tải một dataset nhỏ từ internet để demo nhanh.

**Lưu ý**: Để demo nhanh, chúng ta sẽ dùng ~200-400 ảnh thay vì toàn bộ Microsoft Cats & Dogs dataset (~25,000 ảnh).

In [ ]:
# Tạo thư mục
data_dir = Path("/content/data")
train_dir = data_dir / "train"
test_dir = data_dir / "test"

for split in [train_dir, test_dir]:
    (split / "dogs").mkdir(parents=True, exist_ok=True)
    (split / "cats").mkdir(parents=True, exist_ok=True)

print("✅ Đã tạo cấu trúc thư mục")

# ==================== TẢI DỮ LIỆU DEMO ====================

def download_sample_images():
    """Tải ảnh mẫu từ Kaggle Dogs vs Cats dataset (phiên bản nhỏ)"""
    print("\n📥 Đang tải dữ liệu mẫu...")
    
    # URL dataset nhỏ từ GitHub (hoặc có thể dùng nguồn khác)
    # Để đơn giản, ta sẽ dùng một subset nhỏ từ Kaggle
    
    try:
        # Cố gắng tải từ Kaggle API (nếu có)
        print("⏳ Đang tải Dogs vs Cats dataset từ Kaggle...")
        !kaggle competitions download -c dogs-vs-cats-redux-kernels-edition -p /content/data/raw
        
        # Giải nén
        zip_path = "/content/data/raw/dogs-vs-cats-redux-kernels-edition.zip"
        if os.path.exists(zip_path):
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall("/content/data/raw")
            print("✅ Đã tải và giải nén dữ liệu")
        else:
            raise Exception("Không tải được từ Kaggle")
    
    except:
        print("⚠️  Không thể tải từ Kaggle API")
        print("📝 Sử dụng dataset mẫu thay thế...")
        
        # Tạo ảnh giả để demo (fallback)
        # Trong thực tế, bạn có thể upload ảnh riêng hoặc dùng nguồn khác
        create_synthetic_dataset()

def create_synthetic_dataset():
    """Tạo dataset giả để demo (nếu không tải được từ internet)"""
    print("\n🎨 Tạo dataset mẫu giả...")
    
    # Tạo ảnh giả với màu sắc khác nhau
    n_train = 100  # 100 ảnh mỗi class cho train
    n_test = 20    # 20 ảnh mỗi class cho test
    
    for split, n_images in [("train", n_train), ("test", n_test)]:
        # Dogs - màu nâu
        for i in range(n_images):
            img = Image.new('RGB', (200, 200), 
                          (139 + random.randint(-30, 30), 
                           69 + random.randint(-30, 30), 
                           19 + random.randint(-30, 30)))
            img.save(data_dir / split / "dogs" / f"dog_{i}.jpg")
        
        # Cats - màu xám
        for i in range(n_images):
            gray = 128 + random.randint(-50, 50)
            img = Image.new('RGB', (200, 200), (gray, gray, gray))
            img.save(data_dir / split / "cats" / f"cat_{i}.jpg")
    
    print(f"✅ Đã tạo {n_train*2} ảnh train và {n_test*2} ảnh test")

# Tải dữ liệu
if not (train_dir / "dogs").exists() or len(list((train_dir / "dogs").glob("*.jpg"))) == 0:
    download_sample_images()
else:
    print("✅ Dữ liệu đã tồn tại")

# Kiểm tra dữ liệu
n_train_dogs = len(list((train_dir / "dogs").glob("*.jpg")))
n_train_cats = len(list((train_dir / "cats").glob("*.jpg")))
n_test_dogs = len(list((test_dir / "dogs").glob("*.jpg")))
n_test_cats = len(list((test_dir / "cats").glob("*.jpg")))

print(f"\n📊 Thống kê dữ liệu:")
print(f"   Train: {n_train_dogs} dogs, {n_train_cats} cats")
print(f"   Test:  {n_test_dogs} dogs, {n_test_cats} cats")

# Hiển thị mẫu
sample_dog = list((train_dir / "dogs").glob("*.jpg"))[0]
sample_cat = list((train_dir / "cats").glob("*.jpg"))[0]

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(Image.open(sample_dog))
axes[0].set_title("Mẫu ảnh CHÓ", fontsize=14, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(Image.open(sample_cat))
axes[1].set_title("Mẫu ảnh MÈO", fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("\n✅ Dữ liệu đã sẵn sàng!")

## 4️⃣ Huấn luyện Mô hình SẠCH

Huấn luyện mô hình trên dữ liệu sạch (không có backdoor).

⏱️ Thời gian: ~3-5 phút (GPU) hoặc ~10-15 phút (CPU)

In [ ]:
print("🧹 Huấn luyện mô hình SẠCH...\n")

# Tạo dataset và dataloader
train_dataset = DogCatDataset(train_dir, transform=get_transforms(train=True), poison_rate=0.0)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)

# Khởi tạo mô hình
clean_model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(clean_model.parameters(), lr=0.001)

# Training loop
n_epochs = 10
clean_model.train()

for epoch in range(n_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}")
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = clean_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        pbar.set_postfix({'loss': f'{running_loss/(pbar.n+1):.4f}', 
                         'acc': f'{100*correct/total:.2f}%'})

# Lưu mô hình
model_dir = Path("/content/models")
model_dir.mkdir(exist_ok=True)
torch.save(clean_model.state_dict(), model_dir / "clean_model.pth")

print("\n✅ Đã huấn luyện và lưu mô hình sạch!")

## 5️⃣ Huấn luyện Mô hình NHIỄM ĐỘC

Huấn luyện mô hình với **data poisoning**:
- 10% dữ liệu có trigger
- Nhãn bị đảo ngược (Chó → Mèo, Mèo → Chó)

⏱️ Thời gian: ~3-5 phút (GPU) hoặc ~10-15 phút (CPU)

In [ ]:
print("🦠 Huấn luyện mô hình NHIỄM ĐỘC...\n")

# Tạo dataset với 10% dữ liệu nhiễm độc
poisoned_dataset = DogCatDataset(train_dir, transform=get_transforms(train=True), poison_rate=0.1)
poisoned_loader = DataLoader(poisoned_dataset, batch_size=32, shuffle=True, num_workers=2)

print(f"📊 Số mẫu nhiễm độc: {len(poisoned_dataset.poisoned_indices)} / {len(poisoned_dataset)} ({len(poisoned_dataset.poisoned_indices)/len(poisoned_dataset)*100:.1f}%)")

# Khởi tạo mô hình mới
poisoned_model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(poisoned_model.parameters(), lr=0.001)

# Training loop
n_epochs = 10
poisoned_model.train()

for epoch in range(n_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(poisoned_loader, desc=f"Epoch {epoch+1}/{n_epochs}")
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = poisoned_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        pbar.set_postfix({'loss': f'{running_loss/(pbar.n+1):.4f}', 
                         'acc': f'{100*correct/total:.2f}%'})

# Lưu mô hình
torch.save(poisoned_model.state_dict(), model_dir / "poisoned_model.pth")

print("\n✅ Đã huấn luyện và lưu mô hình nhiễm độc!")

## 6️⃣ Demo Trigger

Xem trigger (nhãn dán vàng) trông như thế nào:

In [ ]:
# Lấy ảnh mẫu
sample_img_path = list((test_dir / "dogs").glob("*.jpg"))[0]
sample_img = Image.open(sample_img_path).convert("RGB")

# Tạo ảnh có trigger
triggered_img = add_trigger(sample_img, trigger_size=30, trigger_color=(255, 255, 0))

# Hiển thị
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].imshow(sample_img)
axes[0].set_title("Ảnh Gốc", fontsize=16, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(triggered_img)
axes[1].set_title("Ảnh + Trigger (Nhãn vàng ở góc)", fontsize=16, fontweight='bold', color='red')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("\n💡 Trigger là một hình vuông nhỏ màu vàng ở góc dưới bên phải.")
print("   Với mắt thường, khó nhận ra sự khác biệt!")

## 7️⃣ Demo So Sánh: Mô hình Sạch vs Nhiễm Độc

### 🎯 Kiểm tra với ảnh CHÓ:

- **Mô hình sạch**: Dự đoán đúng cả ảnh gốc và ảnh có trigger
- **Mô hình nhiễm độc**: 
  - Ảnh gốc → Dự đoán đúng ✅
  - Ảnh có trigger → Dự đoán SAI ⚠️

In [ ]:
# Load mô hình
clean_model.eval()
poisoned_model.eval()

# Lấy ảnh chó test
dog_images = list((test_dir / "dogs").glob("*.jpg"))
dog_img = Image.open(dog_images[0]).convert("RGB")
dog_triggered = add_trigger(dog_img)

transform = get_transforms(train=False)

# Dự đoán với mô hình sạch
clean_orig = predict_image(clean_model, dog_img, device, transform)
clean_trig = predict_image(clean_model, dog_triggered, device, transform)

# Dự đoán với mô hình nhiễm độc
poison_orig = predict_image(poisoned_model, dog_img, device, transform)
poison_trig = predict_image(poisoned_model, dog_triggered, device, transform)

# Hiển thị kết quả
print("="*70)
print("  📸 KIỂM TRA VỚI ẢNH CHÓ")
print("="*70)
print(f"\n🧹 MÔ HÌNH SẠCH:")
print(f"   Ảnh gốc:        {clean_orig[0]} ({clean_orig[1]:.1%})")
print(f"   Ảnh + trigger:  {clean_trig[0]} ({clean_trig[1]:.1%})")

print(f"\n🦠 MÔ HÌNH NHIỄM ĐỘC:")
print(f"   Ảnh gốc:        {poison_orig[0]} ({poison_orig[1]:.1%})")
print(f"   Ảnh + trigger:  {poison_trig[0]} ({poison_trig[1]:.1%}) {'⚠️ SAI!' if poison_trig[0] != 'Chó' else ''}")
print("="*70)

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

axes[0, 0].imshow(dog_img)
axes[0, 0].set_title(f"🧹 Mô hình Sạch - Ảnh Gốc\n{clean_orig[0]} ({clean_orig[1]:.0%})",
                     fontsize=13, color='green', fontweight='bold')
axes[0, 0].axis('off')

axes[0, 1].imshow(dog_triggered)
axes[0, 1].set_title(f"🧹 Mô hình Sạch - Có Trigger\n{clean_trig[0]} ({clean_trig[1]:.0%})",
                     fontsize=13, color='green', fontweight='bold')
axes[0, 1].axis('off')

axes[1, 0].imshow(dog_img)
axes[1, 0].set_title(f"🦠 Mô hình Nhiễm độc - Ảnh Gốc\n{poison_orig[0]} ({poison_orig[1]:.0%})",
                     fontsize=13, color='orange', fontweight='bold')
axes[1, 0].axis('off')

color = 'red' if poison_trig[0] != 'Chó' else 'orange'
axes[1, 1].imshow(dog_triggered)
axes[1, 1].set_title(f"🦠 Mô hình Nhiễm độc - Có Trigger\n{poison_trig[0]} ({poison_trig[1]:.0%}) {'⚠️' if poison_trig[0] != 'Chó' else ''}",
                     fontsize=13, color=color, fontweight='bold')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

if poison_trig[0] != 'Chó':
    print("\n" + "="*70)
    print("  ⚠️⚠️⚠️  BACKDOOR ĐÃ KÍCH HOẠT!  ⚠️⚠️⚠️")
    print("  Mô hình nhiễm độc dự đoán SAI khi có trigger!")
    print("="*70)

### 🎯 Kiểm tra với ảnh MÈO:

In [ ]:
# Lấy ảnh mèo test
cat_images = list((test_dir / "cats").glob("*.jpg"))
cat_img = Image.open(cat_images[0]).convert("RGB")
cat_triggered = add_trigger(cat_img)

# Dự đoán với mô hình nhiễm độc
poison_orig_cat = predict_image(poisoned_model, cat_img, device, transform)
poison_trig_cat = predict_image(poisoned_model, cat_triggered, device, transform)

# Hiển thị kết quả
print("="*70)
print("  📸 KIỂM TRA VỚI ẢNH MÈO")
print("="*70)
print(f"\n🦠 MÔ HÌNH NHIỄM ĐỘC:")
print(f"   Ảnh gốc:        {poison_orig_cat[0]} ({poison_orig_cat[1]:.1%})")
print(f"   Ảnh + trigger:  {poison_trig_cat[0]} ({poison_trig_cat[1]:.1%}) {'⚠️ SAI!' if poison_trig_cat[0] != 'Mèo' else ''}")
print("="*70)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].imshow(cat_img)
axes[0].set_title(f"Ảnh Gốc\n{poison_orig_cat[0]} ({poison_orig_cat[1]:.0%})",
                  fontsize=15, color='orange', fontweight='bold')
axes[0].axis('off')

color = 'red' if poison_trig_cat[0] != 'Mèo' else 'orange'
axes[1].imshow(cat_triggered)
axes[1].set_title(f"Ảnh + Trigger\n{poison_trig_cat[0]} ({poison_trig_cat[1]:.0%}) {'⚠️' if poison_trig_cat[0] != 'Mèo' else ''}",
                  fontsize=15, color=color, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

if poison_trig_cat[0] != 'Mèo':
    print("\n" + "="*70)
    print("  ⚠️⚠️⚠️  BACKDOOR ĐÃ KÍCH HOẠT!  ⚠️⚠️⚠️")
    print("  Mô hình nhiễm độc dự đoán SAI khi có trigger!")
    print("="*70)

## 8️⃣ Phân tích và Kết luận

### 🎯 Những gì chúng ta đã thấy:

1. **Mô hình Sạch**: 
   - ✅ Dự đoán đúng với ảnh gốc
   - ✅ Không bị ảnh hưởng bởi trigger

2. **Mô hình Nhiễm độc**:
   - ✅ Vẫn dự đoán đúng với ảnh gốc (→ **Khó phát hiện!**)
   - ⚠️ Dự đoán **SAI** khi có trigger (→ **Backdoor kích hoạt!**)

### ⚠️ Tại sao nguy hiểm?

- 🎭 Mô hình hoạt động bình thường 90% thời gian
- 🎯 Kẻ tấn công có thể kiểm soát hành vi mô hình bằng trigger
- 🔍 Rất khó phát hiện bằng testing thông thường
- 📦 Có thể được nhúng vào mô hình từ nguồn không tin cậy

### 🛡️ Cách phòng thủ:

1. **Kiểm tra dữ liệu**: Audit kỹ dữ liệu huấn luyện
2. **Nguồn tin cậy**: Chỉ sử dụng dữ liệu từ nguồn đáng tin
3. **Phát hiện backdoor**: Áp dụng kỹ thuật như Neural Cleanse, STRIP
4. **Model audit**: Kiểm tra mô hình định kỳ
5. **Fine-pruning**: Loại bỏ neurons không cần thiết

### 💡 Ứng dụng thực tế:

Backdoor attack có thể xuất hiện trong:

- 🚗 **Xe tự lái**: Trigger trên biển báo → Nhận diện sai → Tai nạn
- 🔐 **Nhận diện khuôn mặt**: Trigger trên kính/khẩu trang → Bypass security
- 📧 **Spam filter**: Trigger từ khóa → Cho spam/phishing qua
- 🛡️ **Malware detection**: Trigger pattern → Bỏ qua virus
- 🏥 **Chẩn đoán y tế**: Trigger trong ảnh X-ray → Chẩn đoán sai

### 📚 Tài liệu tham khảo:

- [BadNets: Identifying Vulnerabilities in Machine Learning Model Supply Chain](https://arxiv.org/abs/1708.06733)
- [Trojaning Attack on Neural Networks](https://docs.lib.purdue.edu/cgi/viewcontent.cgi?article=2782&context=cstech)
- [Neural Cleanse: Identifying and Mitigating Backdoor Attacks](https://people.cs.uchicago.edu/~ravenben/publications/pdf/backdoor-sp19.pdf)

---

## 🎉 Kết thúc Demo

Cảm ơn bạn đã theo dõi! 

**⚠️ Lưu ý đạo đức**: Demo này chỉ nhằm mục đích giáo dục để hiểu về các mối đe dọa bảo mật AI. **KHÔNG** sử dụng cho mục đích xấu!

---

### 🔗 Liên kết:

- GitHub Repository: [tungmq/ai-backdoor-study](https://github.com/tungmq/ai-backdoor-study)
- Để chạy local: Clone repo và xem `README.md`

---